In [1]:
import sys
sys.path.insert(0, '../')

import numpy as np
from src.time_tagger import TimeTaggerController, TT_Simulator, two_particle_states
from src.kinetic_mount_controller import KineticMountControl
from src.osc import OSCCommunicator, OSCTarget
import time
import threading

# Setup Mount Controller

In [2]:
# only run if not already created
if 'KMC' not in globals():  
    KMC = KineticMountControl(number_of_devices=2, address_search_depth=0)
    #KMC.swap_alice_bob()


ASSIGNING CONTROLLERS
---------------------------------------------------------------
Ports Found: ['COM8', 'COM9']
---------------------------------------------------------------
Device succesfully connected (Rotation Mount) (Address: 0)
Device succesfully connected (Rotation Mount) (Address: 0)


In [3]:
# Init the Time Tagger and the Kinetic Mount Controller
TTC = TimeTaggerController()

# NOTE Look at the trace, sucessively block each port and assign the channels numbers in the next cell
# NOTE afterwards stop and close the graphic
# TTC.displayCountTraces()

In [4]:
# connect KMC and TTC
TTC.setKineticMountController(KMC)
# set alice channels
TTC.set_alice_reflection_channel(2)
TTC.set_alice_transmission_channel(4)
# set bob channels
TTC.set_bob_reflection_channel(3)
TTC.set_bob_transmission_channel(1)


# set bob home offset
KMC.bob.set_home_offset(-43 % 360)
KMC.bob.home()
# set alice home offset
KMC.alice.set_home_offset(9)
KMC.alice.home()



In [5]:
_ = TTC.performDelayAdjustment(integration_time=10)

Delays Before Correction
Alice_T :     0 ps 	/     0.0 mm
Alice_R :   875 ps 	/   262.5 mm
Bob_T   :   -12 ps 	/    -3.6 mm
Bob_R   :  -599 ps 	/  -179.7 mm

Delays After Correction
Alice_T :     0 ps 	/     0.0 mm
Alice_R :    25 ps 	/     7.5 mm
Bob_T   :     9 ps 	/     2.7 mm
Bob_R   :    16 ps 	/     4.8 mm


In [6]:
KMC.rotate_simulataneously(0, 0)

In [7]:
n_values = 300
trace_width = 2
TTC.displayCountTraces(n_values=n_values, trace_width=trace_width, plot_title='<b>Raw Counts<b>')
TTC.displayCoincidenceTraces(coincidence_window_SI=0.5e-9, n_values=n_values, trace_width=trace_width, plot_title='<b>Coincidences<b>')

Output()

Output()

No such comm: 086ffc9b14b84897af54d24707f8da29
No such comm: 5c5fd711294f46ed9cdc6740971af55e


In [7]:
KMC.rotate_simulataneously(0, 0)

In [8]:
# TODO
# coincidence channel handling and destruction
#TTC.coincidences_vchannels.getChannels()

In [8]:
# TODO This kills the coincidence channels in the visualisation. Needs to be worked on
idealAngles = [0, -22.5, -11.25, 11.25] # for triplett |HH> + |VV> initial state
TTC.measureS(CHSH_angles=idealAngles, integration_time_per_basis_setting_SI=1, TTSimulator=None, coincidence_window_SI=0.5e-9)



corr[a,b] = 0.8238391574916227
	N[|T,T>]=652
	N[|T,R>]=130
	N[|R,T>]=54
	N[|R,R>]=1253

corr[a,B] = 0.4493700419972002
	N[|T,T>]=523
	N[|T,R>]=410
	N[|R,T>]=180
	N[|R,R>]=1030

corr[A,b] = 0.32367518109035454
	N[|T,T>]=533
	N[|T,R>]=498
	N[|R,T>]=389
	N[|R,R>]=1203

corr[A,B] = -0.7844036697247706
	N[|T,T>]=65
	N[|T,R>]=1507
	N[|R,T>]=827
	N[|R,R>]=217

S = abs(corrs[0,0] + corrs[0,1] + corrs[1,0] - corrs[1,1]) = 2.3812880503039477


# OSC Server Setup


In [9]:
my_ip = '192.168.0.2'
my_port = 7401 
clemens_port = 7401
visual_port = 7401

if 'OSC' not in globals():
    OSC = OSCCommunicator(my_ip=my_ip, my_port=my_port)

Myself = OSCTarget(ip=my_ip, port=my_port, name='Myself')
Clemens = OSCTarget(ip='192.168.0.3', port=clemens_port, name='Clemens')
Visuals = OSCTarget(ip=my_ip, port=visual_port, name='Visuals')

# Start the server
OSC.start_server()

OSC Server Started. Serving on ('192.168.0.2', 7401)
----------------------------------------------------


In [10]:
# Send measurements
measurement_results = [1, 2, 3, 4]
OSC.send_measurement(target=Myself, measurement_results=measurement_results)


Measurement [1, 2, 3, 4] sent to Myself


Received measurement: (1, 2, 3, 4)


In [11]:
# Global flag to control the loop
performance_running = False
resultTranslate = {0 : [1, 1],
                   1 : [1, -1],
                   2 : [-1, 1],
                   3 : [-1, -1]}
CHSH_angles = [0, -22.5, -11.5, 11.5]

def performance_cycle():
    """The function you want to execute repeatedly."""
    global performance_running
    
    time_of_last_visual = time.time()
    while performance_running:
        theta_a = np.random.choice(CHSH_angles[0:2])
        theta_b = np.random.choice(CHSH_angles[2:4])
        measurement_results = TTC.get_single_measurement(theta_a, theta_b)
        resultClemensFormat = [1 if theta_a == CHSH_angles[0] else 2, 1 if theta_b == CHSH_angles[2] else 2 ,*resultTranslate[measurement_results]]

        OSC.send_measurement(Clemens, resultClemensFormat)

        #if time.time() - time_of_last_visual > 7:
        #    OSC.send_visuals(Visuals, resultClemensFormat)
        #    time_of_last_visual = time.time()


def start_performance():
    """Starts the task in a separate thread."""
    global performance_running
    performance_running = True  # Ensure the loop is enabled
    task_thread = threading.Thread(target=performance_cycle, daemon=True)
    task_thread.start()

def stop_performance():
    """Signals the task to stop gracefully."""
    global performance_running
    performance_running = False
    print('performance stopped...')


In [16]:
stop_performance()

performance stopped...


In [15]:
start_performance()

Measurement [2, 2, 1, -1] sent to Clemens
Measurement [1, 2, -1, -1] sent to Clemens
Measurement [2, 2, 1, -1] sent to Clemens
Measurement [1, 1, 1, 1] sent to Clemens
Measurement [1, 2, -1, -1] sent to Clemens
Measurement [1, 2, 1, -1] sent to Clemens
Measurement [1, 1, -1, -1] sent to Clemens
Measurement [1, 1, -1, -1] sent to Clemens
Measurement [2, 2, -1, 1] sent to Clemens
Measurement [2, 1, 1, 1] sent to Clemens
Measurement [2, 2, -1, 1] sent to Clemens
Measurement [1, 1, -1, -1] sent to Clemens
Measurement [2, 2, -1, 1] sent to Clemens
Measurement [1, 2, -1, -1] sent to Clemens
Measurement [1, 1, -1, -1] sent to Clemens
Measurement [2, 1, -1, -1] sent to Clemens
Measurement [1, 2, -1, -1] sent to Clemens
Measurement [1, 1, -1, -1] sent to Clemens
Measurement [2, 2, -1, -1] sent to Clemens
Measurement [2, 1, -1, 1] sent to Clemens
Measurement [2, 2, 1, -1] sent to Clemens
Measurement [1, 2, -1, 1] sent to Clemens
Measurement [2, 2, 1, -1] sent to Clemens
Measurement [1, 1, -1, -1